In [ ]:
import nltk
import numpy as np
nltk.download('punkt')

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
def stem(word):
  return stemmer.stem(word.lower())

In [ ]:
def bag_of_words(tokenized_sentence,all_words):
  tokenized_sentence = [stem(w) for w in tokenized_sentence]
  bag = np.zeros(len(all_words),dtype=np.float32)
  for idx,w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx]=1.0

  return bag

In [ ]:
words = ["organize","organizes","organizing"]
stemmed_words=[stem(word) for word in words]
stemmed_words

['organ', 'organ', 'organ']

# **Train - Data**

In [ ]:
import json
with open('/content/intents (1).json','r') as f:
  intents=json.load(f)

all_words=[]
tags=[]
xy=[]
for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w,tag))

ignore_words=['?','!','.',',']
all_words=[stem(w) for w in all_words if w not in ignore_words]
print(all_words)

['hi', 'hey', 'how', 'are', 'you', 'is', 'anyon', 'there', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'goodby', 'thank', 'thank', 'you', 'that', "'s", 'help', 'thank', "'s", 'a', 'lot', 'which', 'item', 'do', 'you', 'have', 'what', 'kind', 'of', 'item', 'are', 'there', 'what', 'do', 'you', 'sell', 'do', 'you', 'take', 'credit', 'card', 'do', 'you', 'accept', 'mastercard', 'can', 'i', 'pay', 'with', 'paypal', 'are', 'you', 'cash', 'onli', 'how', 'long', 'doe', 'deliveri', 'take', 'how', 'long', 'doe', 'ship', 'take', 'when', 'do', 'i', 'get', 'my', 'deliveri', 'tell', 'me', 'a', 'joke', 'tell', 'me', 'someth', 'funni', 'do', 'you', 'know', 'a', 'joke']


In [ ]:
all_words=sorted(set(all_words))
tags=sorted(set(tags))
print(all_words,'\n',tags)

["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you'] 
 ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [ ]:
import numpy as np

x_train = []
y_train = []
for (pattern_sentence,tag) in xy:
  bag = bag_of_words(pattern_sentence,all_words)
  x_train.append(bag)

  label = tags.index(tag)
  y_train.append(label)


x_train = np.array(x_train)
y_train = np.array(y_train)


In [ ]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(x_train)
    self.x_data = x_train
    self.y_data = y_train

  #dataset[idx]
  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.n_samples



batch_size=8

dataset=ChatDataset()
train_loader=DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,num_workers=2)

# **Model**

In [ ]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super().__init__()
    self.l1=nn.Linear(input_size,hidden_size)
    self.l2=nn.Linear(hidden_size,hidden_size)
    self.l3=nn.Linear(hidden_size,num_classes)
    self.relu=nn.ReLU()

  def forward(self,x):
    out=self.l1(x)
    out=self.relu(out)
    out=self.l2(out)
    out=self.relu(out)
    out=self.l3(out)
    return out

In [ ]:
hidden_size=8
output_size=len(tags)
input_size= len(x_train[0])

model = NeuralNet(input_size,hidden_size,output_size)

In [ ]:
model

NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [ ]:
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        #words = words.to(device)
        labels = labels.to(dtype=torch.long)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch [100/1000], Loss: 1.2249
Epoch [200/1000], Loss: 0.1970
Epoch [300/1000], Loss: 0.0143
Epoch [400/1000], Loss: 0.0283
Epoch [500/1000], Loss: 0.0126
Epoch [600/1000], Loss: 0.0013
Epoch [700/1000], Loss: 0.0008
Epoch [800/1000], Loss: 0.0006
Epoch [900/1000], Loss: 0.0007
Epoch [1000/1000], Loss: 0.0006
final loss: 0.0006


In [ ]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


# **Chat**

In [ ]:
import json
import random
import torch

#from model import NeuralNet
#from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('/content/intents (1).json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "DABBI"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: Sidaa mathad ole...")

Let's chat! (type 'quit' to exit)
You: hie
DABBI: Sidaa mathad ole...
You: who are you
DABBI: Sidaa mathad ole...
You: hello how are you
DABBI: yak ole elli yak bandi
You: what do you sell
DABBI: Yene edru kodalla
You: ndjfnkjr
DABBI: Sidaa mathad ole...
You: loude
DABBI: Sidaa mathad ole...
You: Hello
DABBI: Yen bek ole ninga?
You: prteek bolimaga
DABBI: Sidaa mathad ole...
You: what are the payments options
DABBI: Hatya :-)
You: how do i pay you
DABBI: Sidaa mathad ole...
You: how do i pay you
DABBI: Sidaa mathad ole...
You: do you accept credit card
DABBI: Peche pant kolthu card marle
You: quit
